## Simple code to test the servos

Inspired by AIY servo examples.

In [1]:
from IPython.display import display, HTML, IFrame
from aiy.vision.streaming.server import StreamingServer
from picamera import PiCamera
import ipywidgets.widgets as widgets
from IPython.display import display, HTML, IFrame

from gpiozero import Servo
from aiy.pins import PIN_A, PIN_B

import time

In [2]:
class Robot():
    def __init__(self, pin_right=PIN_A, pin_left=PIN_B, zero_left=-0.05, zero_right=-0.05):
        self.pin_right = pin_right
        self.pin_left = pin_left
        self.zero_right = zero_right
        self.zero_left = zero_left
        
        self.motor_left = Servo(self.pin_right)
        self.motor_right = Servo(self.pin_left)
    
    def forward(self, speed=0.5):
        self.motor_left.value = speed
        self.motor_right.value = -speed

    def backward(self, speed=0.5):
        self.motor_left.value = -speed
        self.motor_right.value = speed

    def left(self, speed=0.5):
        self.motor_left.value = speed
        self.motor_right.value = self.zero_right

    def right(self, speed=0.5):
        self.motor_left.value = self.zero_left
        self.motor_right.value = -speed

    def stop(self):
        self.motor_left.value = self.zero_left
        self.motor_right.value = self.zero_right
        

In [3]:
bot = Robot()

/usr/lib/python3/dist-packages/gpiozero/output_devices.py:1533: PWMSoftwareFallback: To reduce servo jitter, use the pigpio pin factory.See https://gpiozero.readthedocs.io/en/stable/api_output.html#servo for more info
  'To reduce servo jitter, use the pigpio pin factory.'


In [4]:
camera = PiCamera(sensor_mode=4, resolution=(410, 308))
server = StreamingServer(camera, bitrate=75000)

In [5]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

speed_slider = widgets.FloatSlider(value=0.5, min=0.2, max=1.0, step=0.1, description='Speed:',
                                   disabled=False, continuous_update=False, orientation='vertical',
                                   readout=True, readout_format='.1f')

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])

# video frame
video = widgets.HTML(value="<iframe src='http://192.168.0.138:4664/' width=450 height=380>")

display(widgets.HBox([video,controls_box,speed_slider]))

In [6]:
def stop(change):
    bot.stop()
    
def step_forward(change):
    bot.forward(speed_slider.value)
    time.sleep(0.5)
    bot.stop()

def step_backward(change):
    bot.backward(speed_slider.value)
    time.sleep(0.5)
    bot.stop()

def step_left(change):
    bot.left(speed_slider.value)
    time.sleep(0.5)
    bot.stop()

def step_right(change):
    bot.right(speed_slider.value)
    time.sleep(0.5)
    bot.stop()

In [7]:
# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

In [ ]:
server.close()
camera.close()